<a href="https://colab.research.google.com/github/Tara-Lightner/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-14 19:50:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.36MB/s    in 0.2s    

2022-07-14 19:50:25 (6.36 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [5]:
df2 = df.filter(df["total_votes"]>=20)
df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [6]:
df3 = df2.filter(df2["helpful_votes"]/df2["total_votes"]>=0.5)
df3.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [7]:
paid = df3.filter(df3["vine"]=="Y")
unpaid = df3.filter(df3["vine"]=="N")
paid.describe().show()
unpaid.describe().show()

+-------+-----------+-------------------+--------------+----------+--------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|        customer_id|     review_id|product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+-------------------+--------------+----------+--------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|  count|       1080|               1080|          1080|      1080|                1080|                1080|            1080|              1080|              1080|             1080|1080|             1080|               

In [8]:
paid_count = paid.count()
paid_five_star = paid[paid["star_rating"]==5].count()
percentage_paid_five_star = paid_five_star/paid_count
print(paid_count)
print(paid_five_star)
print(percentage_paid_five_star)

1080
454
0.4203703703703704


In [9]:
unpaid_count = unpaid.count()
unpaid_five_star = unpaid[unpaid["star_rating"]==5].count()
percentage_unpaid_five_star = unpaid_five_star/unpaid_count
print(unpaid_count)
print(unpaid_five_star)
print(percentage_unpaid_five_star)

49673
23043
0.463893865882874
